# Common functions

So far, only contains what is expected to be a heavily reused load-and-parse method.

In [ ]:
public T[] LoadAndParse<T>(string path, Func<string, T> parse = null) {  
    parse = parse ?? (p => (T) Convert.ChangeType(p, typeof(T)));
    var rawInput = System.IO.File.ReadAllLines(path);
    return rawInput
    .Where(p=>!string.IsNullOrWhiteSpace(p))
    .Select(parse)
    .ToArray();
}

# Day 1 - Sonar Sweep

Fairly straightforward implementation of moving sum, with the Part 1 puzzle being the special case of a moving average of 1 item.

Solution is strictly O(n)

The solution below leverages the fact that the window can be advanced with minimum calculations by subtracting the first item of the old window (which is about to exit the window) followed by adding the new element about to be added to the window. 

In this way, we can maintain a window of arbitrary size with just one addition and one subtraction pr. element.

In [ ]:
var input = LoadAndParse<int>(@"day1_input.txt");

Console.WriteLine($"Part 1: {CountIncrements(MovingSums(input, 1))}");
Console.WriteLine($"Part 2: {CountIncrements(MovingSums(input, 3))}");    

int CountIncrements(IEnumerable<int> source)
    => source.Zip(source.Skip(1)).Count(p=>p.Item1 < p.Item2);

IEnumerable<int> MovingSums(IEnumerable<int> source, int windowSize) {
    var initial = source.Take(windowSize).Sum();
    yield return initial;

    //Start a double iterator on the source collection, separated by windowSize elements
    foreach (var pair in source.Zip(source.Skip(windowSize))){

        //When advancing the window, remove the start element of the old window, and add the last element of the new window
        initial -= pair.Item1; 
        initial += pair.Item2;
        yield return initial;
    }
}

Part 1: 1564
Part 2: 1611


# Day 2: Dive!

In [ ]:
enum CommandDirection{ forward, down, up }
record Command (CommandDirection Direction, int Amount );
record Position(int Horizontal, int Depth);
record PositionWithAim(int Horizontal, int Depth, int Aim);

var input =  LoadAndParse<Command>(@"day2_input.txt", p=>{
    var particles = p.Split(" ");
    var direction = Enum.Parse<CommandDirection>(particles[0]);
    var amount = Int32.Parse(particles[1]);
    return new Command(direction, amount);
});

var part1position =new Position(0,0);

foreach(var command in input){
    part1position = command.Direction switch {
        CommandDirection.forward => part1position with {Horizontal = part1position.Horizontal + command.Amount },
        CommandDirection.down => part1position with {Depth = part1position.Depth + command.Amount },
        CommandDirection.up => part1position with {Depth = part1position.Depth - command.Amount },
        _=>part1position
    };
}

var part2position = new PositionWithAim(0,0,0);
foreach(var command in input){
    part2position = command.Direction switch {
        CommandDirection.forward => part2position with {Horizontal = part2position.Horizontal + command.Amount, Depth = part2position.Depth + part2position.Aim * command.Amount},
        CommandDirection.down => part2position with {Aim = part2position.Aim + command.Amount },
        CommandDirection.up => part2position with {Aim = part2position.Aim - command.Amount },
        _=>part2position
    };
}

Console.WriteLine($"Part 1: {part1position.Horizontal * part1position.Depth} ({part1position})");
Console.WriteLine($"Part 2: {part2position.Horizontal * part2position.Depth} ({part2position})");

Part 1: 1250395 (Position { Horizontal = 1909, Depth = 655 })
Part 2: 1451210346 (PositionWithAim { Horizontal = 1909, Depth = 760194, Aim = 655 })


In [ ]:
var input = LoadAndParse<string>(@"day3_input.txt");
var bitmask = (1<<input[0].Length)-1; 
int gamma = 0;

for(int i = 0; i < input[0].Length; i++){
    gamma <<= 1;
    if(input.Count(p=>p[i]=='1') > input.Length/2)
        gamma++;
}

var epsilon = ~gamma & bitmask;

Console.WriteLine($"Gamma: {gamma}");
Console.WriteLine($"Epsilon: {epsilon}");
Console.WriteLine($"Part 1: {gamma * epsilon}");

string Reduce(IEnumerable<string> source, char target)
{
    var limit = source.First().Length;
    for(int i = 0; i < limit; i++){            
        if(source.Count(p => p[i] == target) == source.Count(p => p[i] != target))
            source = source.Where(p=>p[i] == target).ToArray();
        else if(source.Count(p=>p[i]==target) > source.Count()/2)
            source = source.Where(p=>p[i] == '1').ToArray();
        else 
            source = source.Where(p=>p[i] == '0').ToArray();                

        if(source.Count() == 1)
            return source.Single();
    }

    return source.Single();        
}

var O2 = Convert.ToInt32(Reduce(input, '1'), 2);
var CO2 = Convert.ToInt32(Reduce(input, '0'), 2);

Console.WriteLine($"O2: {O2}");
Console.WriteLine($"CO2: {CO2}");
Console.WriteLine($"Part 2: {O2 * CO2}");

Gamma: 2566
Epsilon: 1529
Part 1: 3923414
O2: 2919
CO2: 2005
Part 2: 5852595


# --- Day 4: Giant Squid ---

In [ ]:
class BingoBoard{
    int[,] Numbers = new int[5,5];
    bool[,] Marks = new bool[5,5];
    public static BingoBoard Parse(string[] source) {
        BingoBoard res = new();
        for(int y = 0; y < source.Length; y++) {
            var items = source[y].Split(" ", StringSplitOptions.RemoveEmptyEntries).Select(Int32.Parse).ToArray();
            for(int x = 0; x < items.Length; x++)
                res.Numbers[x,y]=items[x];
        }    
        return res;
    }   
    public void Mark(int entry) {
        for(int x = 0; x < 5; x++)
        for(int y = 0; y < 5; y++)
            if(Numbers[x,y] == entry)
                Marks[x,y] = true;            
    }
    public bool Bingo()
        => Enumerable.Range(0, 5).Any(x=>Enumerable.Range(0, 5).All(y => Marks[x,y]))
        || Enumerable.Range(0,5).Any(y=>Enumerable.Range(0, 5).All(x => Marks[x,y]));
    public int Score()
        => Numbers.Cast<int>().Zip(Marks.Cast<bool>()).Sum(p=>p.Item2?0:p.Item1);
}

List<BingoBoard> Boards = new();

var input = System.IO.File.ReadAllLines(@"day4_input.txt");
var numbers = input[0].Split(",").Select(Int32.Parse).ToArray();
var boards = input.Skip(1).Chunk(6).Select(p=>BingoBoard.Parse(p.Skip(1).ToArray())).ToArray();

BingoBoard firstWinner = default;
BingoBoard lastWinner = default;

foreach(var number in numbers) {
    foreach(var board in boards) board.Mark(number);    

    if(lastWinner != default) {
        Console.WriteLine($"The board to avoid scored {lastWinner.Score()} with final drawing of {number} for a result of {lastWinner.Score() * number}");        
        return;
    }

    if(firstWinner == default && boards.Count(p=>p.Bingo()) == 1) {
        firstWinner = boards.Single(p=>p.Bingo());   
        Console.WriteLine($"Winner winner, chicken dinner - final board score {firstWinner.Score()} with drawing {number} for a result of {firstWinner.Score() * number}");        
    }

    if(boards.Count(p=>!p.Bingo())==1)
        lastWinner = boards.Single(p=>!p.Bingo());
}

Winner winner, chicken dinner - final board score 766 with drawing 95 for a result of 72770
The board to avoid scored 296 with final drawing of 47 for a result of 13912


<null>